In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import platform
import warnings

# seaborn 사용하기
import seaborn as sns

plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

warnings.filterwarnings('ignore')

In [4]:
data=pd.read_csv('../Data/second_target.csv')

target=data.iloc[:,:13]
feature=data[['시간대_06~11_매출_비율','연령대_20_매출_비율','점포수']]
feature.head()

target=pd.concat([target,feature],axis=1)

food=target[target['서비스_업종_코드_명'].isin(
       ['한식음식점', '중식음식점', '일식음식점', '양식음식점',
          '패스트푸드점', '분식전문점',
          '치킨전문점','제과점','호프-간이주점','커피-음료'])]

food.reset_index(drop=True,inplace=True)

### 순서
1) 2014 - 2021년 데이터를 모두 합친다. <br><br>
2) 업종 분류를 서울시에서 외식업, 서비스업, 소매업으로 나누는데 그중에 외식업을 타겟으로 하겠다. <br><br>
2-1) 32개 데이터 있는 것만 사용 <br><br>
2-2) 상점수 0개인거는 제외 <br><br>
3) 상권 분류 별로 분포 다른지 확인 <br><br>
    - 히스토그램으로 표현 <br><br>
4) 구 별로 분포가 다른지와 이 분포가 정규분포를 따르는지 <br><br>
    - 추가적으로 구 별 업종 코드 별로 나누어서 이 분포가 정규분포를 따르는지 확인 , 데이터 개수 확인 <br><br>
5) 동 별로 분포가 다른지와 이 분포가 정규분포를 따르는지 <br><br>
    - 추가적으로 동 별 업종 코드 별로 나누어서 이 분포가 정규분포를 따르는지 확인 , 데이터 개수 확인 <br><br>
6) 상권 별로 분포가 다른지와 이분포가 정규분포를 따르는지 <br><br>
    - 추가적으로 동 별 업종 코드 별로 나누어서 이 분포가 정규분포를 따르는지 확인 , 데이터 개수 확인 <br><br>

### 정규분포 따르는지 확인하는 방법 
- 데이터의 분포가 너무 밑과 위에 쏠리는 경향이 있다. <br><br>
- 그래서 밑에 10% , 위에 10% 제외하고 실행하도록 하자. <br><br>

### 점포수 0 인거 빼기

In [5]:
food = food[food['점포수'] != 0]

### 32개 데이터 모두 있는 거만 사용하기로 하자.

In [6]:
food_count=food.pivot_table(index=['상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명'],aggfunc='size')
food_count=pd.DataFrame(food_count[food_count == 32]).reset_index()
code=zip(food_count['상권_코드'],food_count['서비스_업종_코드'])
food_temp = food[food[['상권_코드','서비스_업종_코드']].apply(tuple,1).isin(code)]
food_temp_count=food_temp.pivot_table(index=['상권_코드', '상권_코드_명', '서비스_업종_코드', '서비스_업종_코드_명'],aggfunc='size')
sum(food_temp_count != 32)

0

In [6]:
food = food_temp

In [7]:
food.to_csv('../Data/food.csv',index=False)

In [5]:
df = pd.read_csv('../Data/golmok.csv')
df

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,시군구코드,시군구명,행정동_코드,행정동_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,시간대_06~11_매출_비율,연령대_20_매출_비율,점포수
0,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100001,한식음식점,316763168,2,15,16
1,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100002,중식음식점,39791852,0,33,1
2,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100003,일식음식점,60325474,0,8,2
3,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100004,양식음식점,199448496,0,28,3
4,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100005,제과점,165688274,24,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200706,2020,3,A,골목상권,11740,강동구,11740650,성내2동,1001010,풍성로37가길,CS100009,호프-간이주점,43457012,0,9,4
200707,2020,3,A,골목상권,11740,강동구,11740650,성내2동,1001010,풍성로37가길,CS100010,커피-음료,28339441,29,20,5
200708,2020,4,A,골목상권,11740,강동구,11740650,성내2동,1001010,풍성로37가길,CS100001,한식음식점,130416139,1,19,12
200709,2020,4,A,골목상권,11740,강동구,11740650,성내2동,1001010,풍성로37가길,CS100009,호프-간이주점,66971298,0,7,4


In [9]:
food

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,시군구코드,시군구명,행정동_코드,행정동_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,시간대_06~11_매출_비율,연령대_20_매출_비율,점포수
0,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100001,한식음식점,316763168,2,15,16
3,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100004,양식음식점,199448496,0,28,3
5,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100006,패스트푸드점,42106127,0,32,2
6,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100008,분식전문점,297024864,0,35,9
7,2021,1,A,골목상권,11110,종로구,11110600,가회동,1000001,계동길,CS100010,커피-음료,764429482,5,28,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307152,2020,4,R,전통시장,11740,강동구,11740650,성내2동,1001488,성내골목시장,CS100001,한식음식점,395278514,3,6,17
307153,2020,4,R,전통시장,11740,강동구,11740650,성내2동,1001488,성내골목시장,CS100005,제과점,53740395,9,10,3
307154,2020,4,R,전통시장,11740,강동구,11740650,성내2동,1001488,성내골목시장,CS100007,치킨전문점,51341345,0,15,4
307155,2020,4,R,전통시장,11740,강동구,11740650,성내2동,1001488,성내골목시장,CS100008,분식전문점,145772026,5,19,5


In [7]:
from outlier import removeTen , thirtyTwo

gwanak_test = removeTen(df,'분기당_매출_금액',10)
# gwanak_test = thirtyTwo(gwanak_test)

In [ ]:
city = pd.read_csv('../Data/salary_remove.csv')
city.head()

In [10]:
gender = gwanak_test[gwanak_test.기준_년_코드 == 2021]

In [12]:
gender

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,시군구코드,시군구명,행정동_코드,행정동_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,시간대_06~11_매출_비율,연령대_20_매출_비율,점포수
181029,2021,4,A,골목상권,11680,강남구,11680580,삼성1동,1000935,테헤란로81길,CS100001,한식음식점,14833785541,0,14,78
181013,2021,2,A,골목상권,11680,강남구,11680580,삼성1동,1000935,테헤란로81길,CS100001,한식음식점,12728439909,0,13,74
181005,2021,1,A,골목상권,11680,강남구,11680580,삼성1동,1000935,테헤란로81길,CS100001,한식음식점,10817387846,0,6,75
126853,2021,4,A,골목상권,11530,구로구,11530540,구로3동,1000644,디지털로32길,CS100001,한식음식점,10375074010,1,29,90
126833,2021,2,A,골목상권,11530,구로구,11530540,구로3동,1000644,디지털로32길,CS100001,한식음식점,9553177232,1,28,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156245,2021,2,A,골목상권,11620,관악구,11620775,난곡동,1000809,법원단지길,CS100003,일식음식점,29580985,0,23,1
156872,2021,3,A,골목상권,11620,관악구,11620765,미성동,1000798,남부순환로172길,CS100008,분식전문점,29559253,0,2,4
167771,2021,2,A,골목상권,11650,서초구,11650621,방배4동,1000853,동광로12길,CS100002,중식음식점,29557779,0,9,1
153579,2021,2,A,골목상권,11620,관악구,11620695,신림동,1000785,관천로22길,CS100003,일식음식점,29554994,0,25,2


In [11]:
pd.pivot_table(gender, index = ['상권_코드', '상권_코드_명', '기준_분기_코드'], values = ['남성_매출_금액', '여성_매출_금액'], aggfunc = 'mean')

KeyError: '남성_매출_금액'

In [ ]:
pd.pivot_table(gender1, index = ['상권_코드', '상권_코드_명'], values = ['남성_매출_비율', '여성_매출_비율'], aggfunc = 'mean')